# Appendix

In [1]:
# Staying data at different times (in PM)
staying_data = {
          # 2:45 | 4 | 5 PM
    "Entrance": [0, 0, 0],
    "Book Cafe": [5, 7, 8],
    "Outdoors": [2, 6, 11],
    "Seats": [10, 15, 15],
    "Printing Area": [0, 0, 2],
    "PC Zone": [15, 9, 7]
}

## Calculating Average Stay Duration

We first calculate the average duration of stay in each area. This gives us an insight into which areas are most frequented and for how long. The average stay duration, \( \bar{x} \), is calculated as:

$$
\bar{x} = \frac{\sum_{i=1}^{n} x_i}{n}
$$

where \( x_i \) is the stay duration in each interval, and \( n \) is the number of intervals.


In [13]:
# Calculate average for staying data
average_staying = {area: sum(counts)/len(counts) for area, counts in staying_data.items()}
print(f"Average Staying Data: {average_staying}")


Average Staying Data: {'Entrance': 0, 'Book Cafe': 20/3, 'Outdoors': 19/3, 'Seats': 40/3, 'Printing Area': 2/3, 'PC Zone': 31/3}


## In-Transit Data Between Areas

Next, we analyze the movement of people between different areas of the library. This helps us understand the traffic flow and connectivity between different sections.


In [14]:
# In transit data between areas
in_transit_data = {
                        # 2:45 | 4 | 5 PM
    ("Entrance", "Outdoors"): [1, 2, 3],
    ("Entrance", "Book Cafe"): [2, 4, 4],
    ("Entrance", "Seats"): [2, 4, 4],
    ("Outdoors", "Seats"): [1, 2, 3],
    ("Seats", "Book Cafe"): [2, 1, 4],
    ("Seats", "Printing Area"): [1, 2, 5],
    ("Seats", "PC Zone"): [2, 3, 2],
    ("PC Zone", "Printing Area"): [0, 1, 1]
}


## Average In-Transit Data

We calculate the average number of people moving between different areas. This data is crucial for understanding the most and least used pathways in the library.


In [15]:
# Calculate average for in transit data
average_in_transit = {areas: sum(counts)/len(counts) for areas, counts in in_transit_data.items()}
print(f"Average In-Transit Data: {average_in_transit}")


Average In-Transit Data: {('Entrance', 'Outdoors'): 2, ('Entrance', 'Book Cafe'): 10/3, ('Entrance', 'Seats'): 10/3, ('Outdoors', 'Seats'): 2, ('Seats', 'Book Cafe'): 7/3, ('Seats', 'Printing Area'): 8/3, ('Seats', 'PC Zone'): 7/3, ('PC Zone', 'Printing Area'): 2/3}


## Constructing the Traffic Matrix

The traffic matrix represents the average movement and stay duration in each area. It's a crucial component for modeling the foot traffic dynamics in the library.


In [18]:
# Define the final matrix with aggregated data
final_matrix = matrix([
    [0, average_in_transit[("Entrance", "Book Cafe")], average_in_transit[("Entrance", "Outdoors")], average_in_transit[("Entrance", "Seats")], 0, 0],
    [average_in_transit[("Entrance", "Book Cafe")], average_staying["Book Cafe"], 0, average_in_transit[("Seats", "Book Cafe")], 0, 0],
    [average_in_transit[("Entrance", "Outdoors")], 0, average_staying["Outdoors"], average_in_transit[("Outdoors", "Seats")], 0, 0],
    [average_in_transit[("Entrance", "Seats")], average_in_transit[("Seats", "Book Cafe")], average_in_transit[("Outdoors", "Seats")], average_staying["Seats"], average_in_transit[("Seats", "Printing Area")], average_in_transit[("Seats", "PC Zone")]],
    [0, 0, 0, average_in_transit[("Seats", "Printing Area")], average_staying["Printing Area"], average_in_transit[("PC Zone", "Printing Area")]],
    [0, 0, 0, average_in_transit[("Seats", "PC Zone")], average_in_transit[("PC Zone", "Printing Area")], average_staying["PC Zone"]]
])

print(f"The Foot Traffic Matrix:\n{final_matrix}")

The Foot Traffic Matrix:
[   0 10/3    2 10/3    0    0]
[10/3 20/3    0  7/3    0    0]
[   2    0 19/3    2    0    0]
[10/3  7/3    2 40/3  8/3  7/3]
[   0    0    0  8/3  2/3  2/3]
[   0    0    0  7/3  2/3 31/3]


## Normalizing the Traffic Matrix

Normalization of the traffic matrix is essential for ensuring that the sum of probabilities in each column equals one. This step is crucial for the accuracy of our model. The normalization is done using the formula:

$$
\text{Normalized Entry} = \frac{\text{Matrix Entry}}{\text{Column Sum}}
$$


In [21]:
# Normalizing the matrix
column_sums = [sum(column) for column in zip(*final_matrix)]
normalized_matrix = matrix([[entry/column_sums[j] for j, entry in enumerate(row)] for row in final_matrix])

print(f"Column Sums:\n{column_sums}\n\n")
print(f"Normalized Matrix:\n{normalized_matrix}")


Column Sums:
[26/3, 37/3, 31/3, 26, 4, 40/3]


Normalized Matrix:
[    0 10/37  6/31  5/39     0     0]
[ 5/13 20/37     0  7/78     0     0]
[ 3/13     0 19/31  1/13     0     0]
[ 5/13  7/37  6/31 20/39   2/3  7/40]
[    0     0     0  4/39   1/6  1/20]
[    0     0     0  7/78   1/6 31/40]


## Eigenvalues and Eigenvectors: Understanding Long-Term Traffic Patterns

Eigenvalues and eigenvectors play a crucial role in understanding the long-term behavior of systems, such as foot traffic in our case. By analyzing the eigenvalues and eigenvectors of the normalized traffic matrix, we can predict how the distribution of people in different areas of the library stabilizes over time.

### The Mathematical Concept

The relationship between a matrix, its eigenvalues, and its eigenvectors is foundational in linear algebra and is expressed by the equation:

$$
A\mathbf{v} = \lambda\mathbf{v}
$$

In this equation:
- \( A \) represents our normalized traffic matrix.
- \( v \) is an eigenvector of \( A \).
- \( λ \) is the corresponding eigenvalue of \( v \).

### Interpretation in Our Context

- **Eigenvectors** \( v )\: These vectors provide the "direction" of the traffic flow in the long term. In our context, each eigenvector represents a possible pattern of how people distribute themselves across different areas of the library.
- **Eigenvalues** \( λ \): These values indicate the "magnitude" or "scaling factor" for each eigenvector. A higher eigenvalue corresponds to a more dominant or likely traffic pattern.

### Why It Matters

- **Steady State**: Over time, the foot traffic distribution tends to align with the eigenvector associated with the largest eigenvalue. This is because, as the matrix is repeatedly applied (akin to passing time), the contributions from other eigenvectors diminish relative to the dominant one.

- **Predictive Power**: By identifying this dominant eigenvector, we can predict the most likely long-term distribution of people across the library's areas, assuming current trends continue.

In [24]:
# Calculating eigenvalues and eigenvectors
eigenvalues, eigenvectors = normalized_matrix.eigenvalues(), normalized_matrix.eigenvectors_right()

print(f"Eigenvalues:\n{eigenvalues}\n\n")

# showing only the relevant eigenvector with the anticipated long-term ration (i.e. eigenvector assosciated with eigenvalue 1)
print(f"Eigenvectors:\n{eigenvectors[0]}")


Eigenvalues:
[1, -0.2249517122674667?, 0.01986095248041045?, 0.4449706883284240?, 0.586927644532651?, 0.7811233727601529?]


Eigenvectors:
(1, [
(1, 37/26, 31/26, 3, 6/13, 20/13)
], 1)


## Long-Term Traffic Patterns

We use the power method by raising the normalized matrix to a high power. This helps us predict the long-term distribution of foot traffic in the library. The long-term traffic pattern is expected to follow the ratio of the eigenvectors because:

$$
\text{M}^{k}v \approx \lambda^{k}v
$$

As \( k \) becomes very large, the vector \( \text{M}^{k}v \) approaches a steady state, indicating the long-term traffic distribution.


In [27]:
M_1000 = normalized_matrix^1000

# Define three initial distributions (vectors summing to one)
vector_startEntrance = vector([1, 0, 0, 0, 0, 0])  # start in the entrance (most realistic)
vector_readersOnly   = vector([0, 1/3, 1/3, 1/3, 0, 0])         # Concentrated in reading (book cafe, seated, outdoors)
vector_libraryServices = vector([0, 0, 0, 1/3, 1/3, 1/3])          # Concentrated in the services, but some reading indoors (Seating, Printing, PC Zone)

print(f"Long-term distribution starting at Entrance:\n{M_1000 * vector_startEntrance.n()}\n\n")

print(f"Long-term distribution for Readers Only:\n{M_1000 * vector_readersOnly.n()}\n\n")

print(f"Long-term distribution for service users (with a little bit of indoor readers):\n{M_1000 * vector_readersOnly.n()}\n\n")


Long-term distribution starting at Entrance:
(0.116071428571429, 0.165178571428571, 0.138392857142857, 0.348214285714286, 0.0535714285714286, 0.178571428571429)


Long-term distribution for Readers Only:
(0.116071428571429, 0.165178571428571, 0.138392857142857, 0.348214285714286, 0.0535714285714286, 0.178571428571429)


Long-term distribution for service users (with a little bit of indoor readers):
(0.116071428571429, 0.165178571428571, 0.138392857142857, 0.348214285714286, 0.0535714285714286, 0.178571428571429)


